In [0]:
storage_account_name = "utsdbetahmid"
storage_account_access_key = "PmPFnutaxDfFU2jXf/LUPiWNmqxD9RDEYxt8R3LrdftAXkiCxuGMx2pJuY3d/gFMg8mFspW5Fx10+AStulpypQ=="
blob_container_name = "bde-at2"

In [0]:
dbutils.fs.mount(
   # source = f'wasbs://{blob_container_name}@{storage_account_name}.blob.core.windows.net',
    #mount_point = f'/mnt/{blob_container_name}/',
    #extra_configs = {'fs.azure.account.key.' + storage_account_name + '.blob.core.windows.net': storage_account_access_key}
)

In [0]:
df_green = spark.read.parquet("/FileStore/green")

In [0]:
df_yellow = spark.read.parquet("/FileStore/yellow")

In [0]:
df_green.count()

Out[40]: 66200401

In [0]:
df_yellow.count()

Out[41]: 663055251

In [0]:
## Exporting green_2015.parquet to csv in Azure Blob

In [0]:
df_green_2015 =  spark.read.parquet("/FileStore/green/green_taxi_2015.parquet")

In [0]:
df_green_2015.write.csv("/mnt/bde-at2/output_csv_file/")

In [0]:
## Data Exploration

In [0]:
df_green.printSchema()

root
 |-- VendorID: long (nullable = true)
 |-- lpep_pickup_datetime: timestamp (nullable = true)
 |-- lpep_dropoff_datetime: timestamp (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- RatecodeID: double (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- ehail_fee: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- payment_type: double (nullable = true)
 |-- trip_type: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)



In [0]:
df_yellow.printSchema()

root
 |-- VendorID: long (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: double (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: double (nullable = true)



In [0]:
df_green.show(5)

+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|VendorID|lpep_pickup_datetime|lpep_dropoff_datetime|store_and_fwd_flag|RatecodeID|PULocationID|DOLocationID|passenger_count|trip_distance|fare_amount|extra|mta_tax|tip_amount|tolls_amount|ehail_fee|improvement_surcharge|total_amount|payment_type|trip_type|congestion_surcharge|
+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|       2| 2015-05-01 00:24:18|  2015-05-01 00:24:39|                 N|       1.0|         146|         146|            1.0|          0.0|        2.5|  0.5|    0.

In [0]:
df_yellow.show(5)

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       1| 2015-01-01 00:11:33|  2015-01-01 00:16:48|            1.0|          1.0|       1.0|                 N|          41|         166|           1|        5.7|  0.5|    0.5|       1.

In [0]:
## Data Cleaning

In [0]:
### Removing Trips finishing before starting time

In [0]:
df_yellow_cleaned = df_yellow.where(df_yellow.tpep_pickup_datetime < df_yellow.tpep_dropoff_datetime)
df_green_cleaned = df_green.where(df_green.lpep_pickup_datetime < df_green.lpep_dropoff_datetime)

In [0]:
## Removing Trips with Negative Speed

In [0]:
import pyspark.sql.functions as F

In [0]:
# Create a Column Duration. We need it to calculate speed
df_yellow_cleaned = df_yellow_cleaned.withColumn('duration_seconds', F.col("tpep_dropoff_datetime").cast("long") - F.col("tpep_pickup_datetime").cast("long"))
df_green_cleaned = df_green_cleaned.withColumn('duration_seconds', F.col("lpep_dropoff_datetime").cast("long") - F.col("lpep_pickup_datetime").cast("long"))

# Remove negative speed trips
df_yellow_cleaned = df_yellow_cleaned.where((df_yellow_cleaned.trip_distance/df_yellow_cleaned.duration_seconds) > 0)
df_green_cleaned = df_green_cleaned.where((df_green_cleaned.trip_distance/df_green_cleaned.duration_seconds) > 0)

In [0]:
## Converting Seconds to Hours for the duration
df_yellow_cleaned = df_yellow_cleaned.withColumn('duration_hours', F.col("duration_seconds")/3600)
df_green_cleaned = df_green_cleaned.withColumn('duration_hours', F.col("duration_seconds")/3600)

In [0]:
df_yellow_cleaned = df_yellow_cleaned.withColumn('speed_mph', F.col("trip_distance")/F.col("duration_hours"))
df_green_cleaned = df_green_cleaned.withColumn('speed_mph', F.col("trip_distance")/F.col("duration_hours"))

In [0]:
# Remove trips with very high speed limits for a taxi (NYC speed limit = 55 mph)

df_yellow_cleaned = df_yellow_cleaned.where(df_yellow_cleaned.speed_mph <= 55)
df_green_cleaned = df_green_cleaned.where(df_green_cleaned.speed_mph <= 55)

In [0]:
# Checking the green 2015 dataframe to understand the range of values of the duration column

df_green_2015 = df_green_2015.withColumn('duration_mins', (F.col("lpep_dropoff_datetime").cast("long") - F.col("lpep_pickup_datetime").cast("long"))/60)
df_green_2015_quantiles = df_green_2015.approxQuantile("duration_mins", [0.01, 0.25, 0.75, 0.99], 0)
df_green_2015_quantiles

Out[12]: [0.3, 6.066666666666666, 16.933333333333334, 55.2]

In [0]:
df_green_2015_quantiles

Out[61]: [0.3, 6.066666666666666, 16.933333333333334, 55.2]

In [0]:
df_green_2015.groupby().max("duration_mins").first()

Out[62]: Row(max(duration_mins)=3155397.5)

In [0]:
# Removing trips that were shorter than 1 minute
df_yellow_cleaned = df_yellow_cleaned.where(df_yellow_cleaned.duration_seconds > 60)
df_green_cleaned = df_green_cleaned.where(df_green_cleaned.duration_seconds > 60)

# Removing trips that were longer than 2.5 hours
df_yellow_cleaned = df_yellow_cleaned.where(df_yellow_cleaned.duration_seconds <= 9000)
df_green_cleaned = df_green_cleaned.where(df_green_cleaned.duration_seconds <= 9000)

In [0]:
#### Removing trips with too long or short distance values

In [0]:
# Checking the range of distance values for trip_distance in df_green_2015
df_green_2015_quantiles = df_green_2015.approxQuantile("trip_distance", [0.05, 0.25, 0.75, 0.99], 0)
df_green_2015_quantiles

Out[65]: [0.47, 1.06, 3.64, 14.5]

In [0]:
# Checking the maximum value of yellow april 2022 dataframe for trip_distance
df_green_2015.groupby().max("trip_distance").first()

Out[66]: Row(max(trip_distance)=830.4)

In [0]:
# Remove trips that were less than 0.45 miles
df_yellow_cleaned = df_yellow_cleaned.where(df_yellow_cleaned.trip_distance >= 0.45)
df_green_cleaned = df_green_cleaned.where(df_green_cleaned.trip_distance >= 0.45)

# Remove trips that were more than 50 miles
df_yellow_cleaned = df_yellow_cleaned.where(df_yellow_cleaned.trip_distance <= 50)
df_green_cleaned = df_green_cleaned.where(df_green_cleaned.trip_distance <= 50)

In [0]:
## Removing trips that started before January 2015 or started after December 2022

In [0]:
# Remove trips that started before January 2015 for green taxis
df_yellow_cleaned = df_yellow_cleaned.where(df_yellow_cleaned.tpep_pickup_datetime >= '2015-01-01 00:00:00')
df_green_cleaned = df_green_cleaned.where(df_green_cleaned.lpep_pickup_datetime >= '2015-01-01 00:00:00')

# Remove trips that started after December 2022 for green taxis
df_yellow_cleaned = df_yellow_cleaned.where(df_yellow_cleaned.tpep_pickup_datetime < '2023-01-01 00:00:00')
df_green_cleaned = df_green_cleaned.where(df_green_cleaned.lpep_pickup_datetime < '2023-01-01 00:00:00')

In [0]:
#Combining Yellow and Green data 

In [0]:
# Change the column names for pickup time and dropoff time for both data frames
df_yellow_cleaned = df_yellow_cleaned.withColumnRenamed("tpep_pickup_datetime", "pickup_datetime")
df_yellow_cleaned = df_yellow_cleaned.withColumnRenamed("tpep_dropoff_datetime", "dropoff_datetime")
df_green_cleaned = df_green_cleaned.withColumnRenamed("lpep_pickup_datetime", "pickup_datetime")
df_green_cleaned = df_green_cleaned.withColumnRenamed("lpep_dropoff_datetime", "dropoff_datetime")

In [0]:
## Viewing the columns and size of the dataframes

In [0]:
print(df_yellow_cleaned.columns)

['VendorID', 'pickup_datetime', 'dropoff_datetime', 'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag', 'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge', 'total_amount', 'congestion_surcharge', 'airport_fee', 'duration_seconds', 'duration_hours', 'speed_mph']


In [0]:

print(df_green_cleaned.columns)

['VendorID', 'pickup_datetime', 'dropoff_datetime', 'store_and_fwd_flag', 'RatecodeID', 'PULocationID', 'DOLocationID', 'passenger_count', 'trip_distance', 'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount', 'ehail_fee', 'improvement_surcharge', 'total_amount', 'payment_type', 'trip_type', 'congestion_surcharge', 'duration_seconds', 'duration_hours', 'speed_mph']


In [0]:
# Create a list of column names that are common to both data frames
common_cols = ['VendorID', 'pickup_datetime',  'passenger_count', 'trip_distance',  'PULocationID', 'DOLocationID', 'tip_amount', 'total_amount', 'duration_seconds', 'duration_hours', 'speed_mph']

# Select only the common columns for both data frames
df_yellow_cleaned = df_yellow_cleaned.select(common_cols)
df_green_cleaned = df_green_cleaned.select(common_cols)

In [0]:
df_green_cleaned.printSchema()

root
 |-- VendorID: long (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- duration_seconds: long (nullable = true)
 |-- duration_hours: double (nullable = true)
 |-- speed_mph: double (nullable = true)
 |-- taxi_color: string (nullable = false)



In [0]:
df_yellow_cleaned.printSchema()

root
 |-- VendorID: long (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- duration_seconds: long (nullable = true)
 |-- duration_hours: double (nullable = true)
 |-- speed_mph: double (nullable = true)
 |-- taxi_color: string (nullable = false)



In [0]:
df_green_cleaned.write.parquet("/FileStore/df_green_cleaned")

In [0]:
df_green_cleaned = spark.read.parquet("/FileStore/df_green_cleaned")

In [0]:
df_yellow_cleaned.write.parquet("/FileStore/df_yellow_cleaned")

In [0]:
# Labelling the datasets with their taxi color

In [0]:
df_yellow_cleaned = df_yellow_cleaned.withColumn("taxi_color", F.lit("yellow"))
df_green_cleaned = df_green_cleaned.withColumn("taxi_color", F.lit("green"))

In [0]:
sample_df_yellow = df_yellow_cleaned.sample(fraction = 0.01, seed = 42)

In [0]:
Combine the yellow and green taxis dataframes
merged_df = sample_df_yellow.union(df_green_cleaned)

In [0]:
merged_df = df_green_cleaned.alias("merged_df")

In [0]:
# Check the schema of the merged dataframe
merged_df.printSchema()

root
 |-- VendorID: long (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- duration_seconds: long (nullable = true)
 |-- duration_hours: double (nullable = true)
 |-- speed_mph: double (nullable = true)
 |-- taxi_color: string (nullable = false)



In [0]:
# Exporting the Combined df to DBFS

In [0]:
type(merged_df)

Out[81]: pyspark.sql.dataframe.DataFrame

In [0]:
taxi_location = spark.read.csv("/FileStore/taxi_zone_lookup.csv", header=True)

In [0]:
combined_df = merged_df.alias("combined_df")

In [0]:
## Joining the Location Lookup Table to the merged dataset

In [0]:
# Joining Location for PickUP (PU) data

In [0]:
combined_df = combined_df.join(taxi_location, 
               combined_df.PULocationID == taxi_location.LocationID, 
               "inner")

In [0]:
combined_df = combined_df.drop(combined_df.LocationID)
combined_df = combined_df.drop(combined_df.Zone)
combined_df = combined_df.drop(combined_df.service_zone)

In [0]:
combined_df = combined_df.withColumnRenamed("Borough", "PUBorough")

In [0]:
# Joining Location for the Drop Off (DO) Data

In [0]:
combined_df = combined_df.join(taxi_location, 
               combined_df.DOLocationID == taxi_location.LocationID, 
               "inner")

In [0]:
# Dropping the redundant Location ID COlumn
combined_df = combined_df.drop(combined_df.LocationID)
combined_df = combined_df.drop(combined_df.Zone)
combined_df = combined_df.drop(combined_df.service_zone)

In [0]:
combined_df = combined_df.withColumnRenamed("Borough", "DOBorough")

In [0]:
combined_df.printSchema()

root
 |-- VendorID: long (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- duration_seconds: long (nullable = true)
 |-- duration_hours: double (nullable = true)
 |-- speed_mph: double (nullable = true)
 |-- taxi_color: string (nullable = false)
 |-- PUBorough: string (nullable = true)
 |-- DOBorough: string (nullable = true)



In [0]:
combined_df.write.parquet("/FileStore/green_yellow_location_combined")

In [0]:
combined_df.createOrReplaceTempView("merged_data_view")

In [0]:
## Read the combined data from DBFS and create a temporary veiw

In [0]:
merged_data = spark.read.parquet("/FileStore/combined_green_yellow_location")

# Create a temporary view for the combined data
merged_data.createOrReplaceTempView("merged_data_view")

In [0]:
#### Part2: Business Questions

In [0]:
# Finding the total number of trips for each year and month
year_month_stats = spark.sql('''
SELECT DATE_FORMAT(pickup_datetime, 'MMM y') AS year_month, count(*) AS total_number_trips
FROM merged_data_view
GROUP BY year_month
''')

In [0]:
# Finding out which day of week had the most trips for each year and month
day_highest_trips = spark.sql('''
WITH trip_ranks AS
(SELECT year_month, day_week, number_trips,
RANK() over (partition by year_month order by number_trips desc) as rk
FROM
(SELECT DATE_FORMAT(pickup_datetime, 'MMM y') AS year_month, 
DATE_FORMAT(pickup_datetime, 'E') AS day_week, count(*) AS number_trips
FROM merged_data_view
GROUP BY year_month, day_week) AS num_trips_day)
SELECT year_month, day_week
FROM trip_ranks
WHERE rk = 1
''')

In [0]:
# Joining the dataframes year_month_stats and day_highest_trips
year_month_stats = year_month_stats.join(day_highest_trips, "year_month", "inner")

In [0]:
# Finding out which hour of day had the most trips for each year and month
hour_highest_trips = spark.sql('''
WITH trip_ranks_hour AS
(SELECT year_month, hour_day, number_trips,
RANK() over (partition by year_month order by number_trips desc) as rk
FROM
(SELECT DATE_FORMAT(pickup_datetime, 'MMM y') AS year_month, 
DATE_FORMAT(pickup_datetime, 'H') AS hour_day, count(*) AS number_trips
FROM merged_data_view
GROUP BY year_month, hour_day) AS num_trips_hour_day)
SELECT year_month, hour_day
FROM trip_ranks_hour
WHERE rk = 1
''')

In [0]:
# Joining the dataframes year_month_stats and hour_highest_trips
year_month_stats = year_month_stats.join(hour_highest_trips, "year_month", "inner")

In [0]:
# Finding the average number of passengers for each year and month
avg_passengers = spark.sql('''
SELECT DATE_FORMAT(pickup_datetime, 'MMM y') AS year_month, avg(passenger_count) as avg_number_passengers
FROM merged_data_view
GROUP BY year_month
''')

In [0]:
# Joining the dataframes year_month_stats and avg_passengers
year_month_stats = year_month_stats.join(avg_passengers, "year_month", "inner")

In [0]:
# Finding the average amount paid per trip for each year and month
avg_amount_paid_per_trip = spark.sql('''
SELECT DATE_FORMAT(pickup_datetime, 'MMM y') AS year_month, avg(total_amount) as avg_amount_paid_per_trip
FROM merged_data_view
GROUP BY year_month
''')

In [0]:
# Joining the dataframes year_month_stats and avg_amount_paid_per_trip
year_month_stats = year_month_stats.join(avg_amount_paid_per_trip, "year_month", "inner")

In [0]:
# Finding the average amount paid per passenger for each year and month
avg_amount_paid_per_passenger = spark.sql('''
SELECT DATE_FORMAT(pickup_datetime, 'MMM y') AS year_month, sum(total_amount)/sum(passenger_count) as avg_amount_paid_per_passenger
FROM merged_data_view
GROUP BY year_month
''')

In [0]:
# Joining the dataframes year_month_stats and avg_amount_paid_per_passenger
year_month_stats = year_month_stats.join(avg_amount_paid_per_passenger, "year_month", "inner")

In [0]:
# Display the year_month_stats dataframe
year_month_stats.show(10)

+----------+------------------+--------+--------+---------------------+------------------------+-----------------------------+
|year_month|total_number_trips|day_week|hour_day|avg_number_passengers|avg_amount_paid_per_trip|avg_amount_paid_per_passenger|
+----------+------------------+--------+--------+---------------------+------------------------+-----------------------------+
|  Apr 2015|           1580592|     Sat|      18|   1.3804618775749846|      14.950140713397484|           10.829810628063083|
|  Apr 2016|           1458480|     Sat|      18|   1.3663972080522186|      14.971400677684295|           10.956843727034418|
|  Apr 2017|           1022159|     Sat|      18|     1.36612210037773|      14.357049705730981|           10.509345908218076|
|  Apr 2018|            758813|     Sun|      18|   1.3653587906374824|      15.816456426110816|            11.58410268023847|
|  Apr 2019|            536977|     Tue|      18|   1.3161163468429673|      18.770194458898914|            15.

In [0]:
## Part2: Q2

In [0]:
spark.sql('''
SELECT taxi_color, round(avg(duration_seconds/60),2) as avg_duration_mins, 
round(approx_percentile(duration_seconds/60, 0.5),2) as median_duration_mins,
round(min(duration_seconds/60),2) as min_duration_mins, 
round(max(duration_seconds/60),2) as max_duration_mins,
round(avg(trip_distance*1.60934),2) as avg_distance_km,
round(approx_percentile(trip_distance*1.60934, 0.5),2) as median_distance_km,
round(min(trip_distance*1.60934),2) as min_distance_km,
round(max(trip_distance*1.60934),2) as max_distance_km,
round(avg((trip_distance*1.60934)/duration_hours),2) as avg_speed_kph,
round(approx_percentile((trip_distance*1.60934)/duration_hours, 0.5),2) as median_speed_kph,
round(min((trip_distance*1.60934)/duration_hours),2) as min_speed_kph,
round(max((trip_distance*1.60934)/duration_hours),2) as max_speed_kph
FROM merged_data_view
GROUP BY taxi_color
''').show()

+----------+-----------------+--------------------+-----------------+-----------------+---------------+------------------+---------------+---------------+-------------+----------------+-------------+-------------+
|taxi_color|avg_duration_mins|median_duration_mins|min_duration_mins|max_duration_mins|avg_distance_km|median_distance_km|min_distance_km|max_distance_km|avg_speed_kph|median_speed_kph|min_speed_kph|max_speed_kph|
+----------+-----------------+--------------------+-----------------+-----------------+---------------+------------------+---------------+---------------+-------------+----------------+-------------+-------------+
|     green|            14.28|               10.98|             1.02|            150.0|           5.04|              3.22|           0.72|          80.47|        20.44|           18.48|          0.3|        88.51|
|    yellow|            14.28|               10.98|             1.02|            150.0|           5.04|              3.22|           0.72|      

In [0]:
## Part 2: Q3

In [0]:
spark.sql('''
SELECT taxi_color, 
DATE_FORMAT(pickup_datetime, 'MMM y') AS year_month,
DATE_FORMAT(pickup_datetime, 'E') AS day_week,
DATE_FORMAT(pickup_datetime, 'H') AS hour_day,
count(*) as total_nubmer_of_trips,
round(avg(trip_distance*1.60934),2) as avg_distance_km,
round(avg(total_amount),2) as avg_amount_paid_per_trip,
round(sum(total_amount),2) as total_amount_paid
FROM merged_data_view
GROUP BY taxi_color, PUBorough,DOBorough, year_month, day_week, hour_day
''').show()

+----------+----------+--------+--------+---------------------+---------------+------------------------+-----------------+
|taxi_color|year_month|day_week|hour_day|total_nubmer_of_trips|avg_distance_km|avg_amount_paid_per_trip|total_amount_paid|
+----------+----------+--------+--------+---------------------+---------------+------------------------+-----------------+
|    yellow|  May 2015|     Fri|       1|                    9|          21.36|                   45.31|            407.8|
|    yellow|  May 2015|     Fri|       2|                    7|           12.7|                   28.27|           197.88|
|    yellow|  May 2015|     Fri|       4|                  132|           4.46|                   12.43|          1640.66|
|     green|  May 2015|     Fri|      12|                  129|           9.58|                   30.97|           3995.7|
|    yellow|  May 2015|     Fri|      13|                   55|          12.59|                   31.74|          1745.78|
|     green|  Ma

In [0]:
## Part 2: Q4

In [0]:
# Finding the percentage of trips where drivers received a tip
spark.sql('''
SELECT (count(*)/(SELECT count(*) FROM merged_data_view))*100 as percentage_trips_received_tip
FROM merged_data_view
WHERE tip_amount != 0
''').show()

+-----------------------------+
|percentage_trips_received_tip|
+-----------------------------+
|            40.97119561399637|
+-----------------------------+



In [0]:
## Part 2: Q5

In [0]:
# Finding the percentage of trips with tips where drivers received a tip of at least $10
spark.sql('''
WITH tips_received as
(SELECT *
FROM merged_data_view
WHERE tip_amount != 0)
SELECT (count(*)/(SELECT count(*) FROM tips_received))*100 as percentage_trips_tips_more_than_five_dollars
FROM tips_received
WHERE tip_amount >= 5
''').show()

+--------------------------------------------+
|percentage_trips_tips_more_than_five_dollars|
+--------------------------------------------+
|                          12.205032370644812|
+--------------------------------------------+



In [0]:
## Part 2: Q6

In [0]:
# Finding average speed, average distance per dollar for 5 bins of durations
spark.sql('''
WITH combined_data_with_duration_bins as
(SELECT *,
CASE
    WHEN (duration_seconds/60) < 5 THEN 'Under 5 mins'
    WHEN (duration_seconds/60) >= 5 and (duration_seconds/60) < 10 THEN 'From 5 mins to 10 mins'
    WHEN (duration_seconds/60) >= 10 and (duration_seconds/60) < 20 THEN 'From 10 mins to 20 mins'
    WHEN (duration_seconds/60) >= 20 and (duration_seconds/60) < 30 THEN 'From 20 mins to 30 mins'
    WHEN (duration_seconds/60) >= 30 and (duration_seconds/60) < 60 THEN 'From 30 mins to 60 mins'
    WHEN (duration_seconds/60) >= 60  THEN 'At least 60 mins'
END as duration_bins
FROM merged_data_view)
SELECT duration_bins, 
round(avg((trip_distance*1.60934)/duration_hours),2) as avg_speed_kph,
round(avg((trip_distance*1.60934)/total_amount),2) as avg_distance_per_dollar
FROM combined_data_with_duration_bins
GROUP BY duration_bins
''').display()

duration_bins,avg_speed_kph,avg_distance_per_dollar
From 5 mins to 10 mins,18.57,0.25
Under 5 mins,20.65,0.19
From 10 mins to 20 mins,20.82,0.32
From 20 mins to 30 mins,22.57,0.37
At least 60 mins,19.84,0.9
From 30 mins to 60 mins,22.68,0.44


In [0]:
## Part2: Q7

In [0]:
# Duration bin to maximize the driver's income
spark.sql('''
WITH combined_data_with_duration_bins as
(SELECT *,
CASE
    WHEN (duration_seconds/60) < 5 THEN 'Under 5 mins'
    WHEN (duration_seconds/60) >= 5 and (duration_seconds/60) < 10 THEN 'From 5 mins to 10 mins'
    WHEN (duration_seconds/60) >= 10 and (duration_seconds/60) < 20 THEN 'From 10 mins to 20 mins'
    WHEN (duration_seconds/60) >= 20 and (duration_seconds/60) < 30 THEN 'From 20 mins to 30 mins'
    WHEN (duration_seconds/60) >= 30 and (duration_seconds/60) < 60 THEN 'From 30 mins to 60 mins'
    WHEN (duration_seconds/60) >= 60  THEN 'At least 60 mins'
END as duration_bins
FROM merged_data_view)
SELECT duration_bins, 
round(sum(total_amount),2) as sum_amount,
round(avg(total_amount),2) as avg_amount,
round(approx_percentile(total_amount, 0.5),2) as median_amount,
round(avg((trip_distance*1.60934)/total_amount),2) as avg_distance_per_dollar_total_amount
FROM combined_data_with_duration_bins
GROUP BY duration_bins
''').display()

duration_bins,sum_amount,avg_amount,median_amount,avg_distance_per_dollar_total_amount
From 5 mins to 10 mins,1.8333253714E8,9.32,8.8,0.25
Under 5 mins,5.516366275E7,6.57,6.3,0.19
From 10 mins to 20 mins,3.3411689859E8,15.59,14.3,0.32
From 20 mins to 30 mins,1.9510024593E8,25.31,23.3,0.37
At least 60 mins,3.200992387E7,61.0,59.3,0.9
From 30 mins to 60 mins,1.8415835201E8,38.59,35.38,0.44
